In [1]:
import numpy as np
import pandas as pd

In [4]:
!pip install opendatasets

# Preprocessing

In [2]:
link = 'https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction'

In [5]:
import opendatasets as od

od.download(link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aiduhanina
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction


100%|██████████| 8.56k/8.56k [00:00<00:00, 11.8MB/s]

In [13]:
df = pd.read_csv('heart-failure-prediction/heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [14]:
# Droping irrelevant
df.dropna(inplace=True)

In [15]:
df['HeartDisease'].value_counts()

,count
HeartDisease,
1,508
0,410


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


## Binarizing the data

In [17]:
def preprocessing(df, features, scales, bounds):
    output = df.copy()
    for scale in scales:
        curr_feats = features[scale]
        output.drop(columns=curr_feats, inplace=True)
        for feat in curr_feats:
            if scale == 'inter':
                curr_bound = bounds[feat]
                for bound in curr_bound:
                    output[f'{feat}_great_{bound}'] = df[feat] >= bound
                    output[f'{feat}_less_{bound}'] = df[feat] <= bound
            elif scale == 'ordinal':
                curr_bound = bounds[feat]
                for bound in curr_bound:
                    output[f'{feat}_great_{bound}'] = df[feat] >= bound
            else:
                if scale == 'nominal':
                    prefix = ''
                    prefix_sep=''
                if scale == 'dichotomic':
                    prefix = f'{feat}'
                    prefix_sep='_'
                dummies = pd.get_dummies(df[feat], prefix=prefix, prefix_sep=prefix_sep)
                output = pd.concat([output, dummies], axis=1)

    return output

### 1

In [18]:
scales = ['inter', 'nominal', 'dichotomic']
features = {'inter': ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak'],
            'nominal': ['ChestPainType', 'RestingECG', 'ST_Slope'],
            'dichotomic': ['Sex', 'FastingBS', 'ExerciseAngina']}
bounds = {'Age': [20, 30, 40, 50, 55, 60, 65, 70, 75, 80],
          'RestingBP': [70, 90, 120, 135, 150, 160, 170, 180],
          'Cholesterol': [100, 150, 200, 240, 300, 400],
          'MaxHR': [60, 70, 80, 90, 100, 110, 120],
          'Oldpeak': [1, 2, 3]}

df_bin = preprocessing(df, features, scales, bounds)

In [19]:
df_bin

,HeartDisease,Age_great_20,Age_less_20,Age_great_30,Age_less_30,Age_great_40,Age_less_40,Age_great_50,Age_less_50,Age_great_55,...,ST,Down,Flat,Up,Sex_F,Sex_M,FastingBS_0,FastingBS_1,ExerciseAngina_N,ExerciseAngina_Y
0,0,True,False,True,False,True,True,False,True,False,...,False,False,False,True,False,True,True,False,True,False
1,1,True,False,True,False,True,False,False,True,False,...,False,False,True,False,True,False,True,False,True,False
2,0,True,False,True,False,False,True,False,True,False,...,True,False,False,True,False,True,True,False,True,False
3,1,True,False,True,False,True,False,False,True,False,...,False,False,True,False,True,False,True,False,False,True
4,0,True,False,True,False,True,False,True,False,False,...,False,False,False,True,False,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,1,True,False,True,False,True,False,False,True,False,...,False,False,True,False,False,True,True,False,True,False
914,1,True,False,True,False,True,False,True,False,True,...,False,False,True,False,False,True,False,True,True,False
915,1,True,False,True,False,True,False,True,False,True,...,False,False,True,False,False,True,True,False,False,True
916,1,True,False,True,False,True,False,True,False,True,...,False,False,True,False,True,False,True,False,True,False


In [20]:
df_bin.to_csv("baseline.csv")

### 2

In [21]:
scales = ['ordinal', 'nominal', 'dichotomic']
features = {'ordinal': ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak'],
            'nominal': ['ChestPainType', 'RestingECG', 'ST_Slope'],
            'dichotomic': ['Sex', 'FastingBS', 'ExerciseAngina']}
bounds = {'Age': [20, 30, 40, 50, 55, 60, 65, 70, 75, 80],
          'RestingBP': [70, 90, 120, 135, 150, 160, 170, 180],
          'Cholesterol': [100, 150, 200, 240, 300, 400],
          'MaxHR': [60, 70, 80, 90, 100, 110, 120],
          'Oldpeak': [1, 2, 3]}

df_bin = preprocessing(df, features, scales, bounds)

In [22]:
df_bin.to_csv("another_baseline.csv")